# Uma análise sobre dados de ameaças cibernéticas

Este relatório apresenta uma análise exploratória das ameaças cibernéticas globais entre 2015 e 2024, com base no conjunto de dados "Global Cybersecurity Threats", bem como testes estatísticos realizados.

## Importação dos dados
Para iniciar a análise, realizei a importação das bibliotecas necessárias e a leitura da base de dados disponibilizada pelo Kaggle.

In [10]:
# Importando as bibliotecas
import pandas as pd
import plotly.express as px
import scipy.stats as stats
from scipy.stats import f_oneway
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Carregando os dados
df = pd.read_csv('../data/Global_Cybersecurity_Threats_2015-2024.csv')

# Visualizando as primeiras linhas
df.head()

,Country,Year,Attack Type,Target Industry,Financial Loss (in Million $),Number of Affected Users,Attack Source,Security Vulnerability Type,Defense Mechanism Used,Incident Resolution Time (in Hours)
0,China,2019,Phishing,Education,80.53,773169,Hacker Group,Unpatched Software,VPN,63
1,China,2019,Ransomware,Retail,62.19,295961,Hacker Group,Unpatched Software,Firewall,71
2,India,2017,Man-in-the-Middle,IT,38.65,605895,Hacker Group,Weak Passwords,VPN,20
3,UK,2024,Ransomware,Telecommunications,41.44,659320,Nation-state,Social Engineering,AI-based Detection,7
4,Germany,2018,Man-in-the-Middle,IT,74.41,810682,Insider,Social Engineering,VPN,68


##  Limpeza e exploração inicial

Para garantir a qualidade dos dados antes da análise, verifiquei o formato, a presença de valores nulos e duplicados.

In [5]:
#Resumo da estrutura de dados
df.info()

#Verificando se existe dados duplicados
print("Dados duplicados: ", df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Country                              3000 non-null   object 
 1   Year                                 3000 non-null   int64  
 2   Attack Type                          3000 non-null   object 
 3   Target Industry                      3000 non-null   object 
 4   Financial Loss (in Million $)        3000 non-null   float64
 5   Number of Affected Users             3000 non-null   int64  
 6   Attack Source                        3000 non-null   object 
 7   Security Vulnerability Type          3000 non-null   object 
 8   Defense Mechanism Used               3000 non-null   object 
 9   Incident Resolution Time (in Hours)  3000 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 234.5+ KB
Dados duplicados:  0


**Validado que a estrutura está adequada, e sem dados nulos ou duplicados.**

A seguir, validei as principais estatísticas para as variáveis numéricas do dataset.

In [6]:
# Estatísticas dos valores númericos
df.describe()

,Year,Financial Loss (in Million $),Number of Affected Users,Incident Resolution Time (in Hours)
count,3000.000000,3000.000000,3000.000000,3000.000000
mean,2019.570333,50.492970,504684.136333,36.476000
std,2.857932,28.791415,289944.084972,20.570768
min,2015.000000,0.500000,424.000000,1.000000
25%,2017.000000,25.757500,255805.250000,19.000000
50%,2020.000000,50.795000,504513.000000,37.000000
75%,2022.000000,75.630000,758088.500000,55.000000
max,2024.000000,99.990000,999635.000000,72.000000


Com base nessas estatísticas, já podemos obter alguns insights:

* A mediana do ano dos incidentes é 2020, com 50% dos ataques ocorrendo entre 2017 e 2022, indicando aumento da frequência ou maior registro de ataques nos últimos anos.
* Os prejuízos financeiros são elevados e variam bastante, com média em torno de 50 milhões de dólares.
* A maioria dos incidentes afeta aproximadamente meio milhão de usuários, o que representa uma escala significativa.
* O tempo médio para resolução dos incidentes é relativamente curto (cerca de 36 horas), porém alguns casos levam até 3 dias para serem resolvidos, podendo causar impactos prolongados, especialmente para setores críticos.

## Análise de Correlação e Testes Estatísticos


### **Correlação entre variáveis númericas**  

Utilizei gráficos de dispersão para identificar possíveis padrões ou relações entre as variáveis.

In [7]:
#Identificando se existe correlação entre as variáveis numéricas
fig = px.scatter_matrix(df.select_dtypes(include=['number']))
fig.update_layout(
    width=800, height=800, font_size=8 
)
fig.show()

Todos os gráficos de dispersão estão com os pontos muito espalhados e sem formar nenhum padrão claro (reta, curva, aglomeração etc.). Isso indica que **não há relação aparente (linear ou não-linear)** entre os pares de variáveis.

Para aprofundar a análise, calculei a matriz de correlação utilizando a função corr(), que, por padrão, aplica o método de Pearson. Esse método mede a força e a direção da relação linear entre duas variáveis, gerando uma pontuação que varia de -1 a +1:

* 0 indica ausência de correlação,
* +1 representa uma correlação positiva perfeita,
* -1 representa uma correlação negativa perfeita.



In [12]:
#Analisando mais a fundo a correlação linear entre os campos numéricos
correlacao = df.select_dtypes(include=['number']).corr()
fig = px.imshow(correlacao,
                text_auto=".5f",
                aspect="auto",
                title='Correlação de Pearson entre variáveis quantitativas',
                color_continuous_scale='RdBu_r')

fig.update_layout(
    title_x=0.5
)

fig.show()

É possível observar que os valores da matriz estão próximos de zero, confirmando a **ausência de correlação linear significativa** entre as variáveis numéricas analisadas.



### Teste ANOVA para variáveis Categóricas x Númericas

Para investigar a existência de diferenças estatisticamente significativas entre os grupos, apliquei o teste ANOVA.
Cada grupo corresponde aos valores da variável numérica para uma categoria específica da variável categórica.

In [6]:
# Listas de variáveis
variaveis_categoricas = ['Country', 'Attack Type', 'Target Industry', 'Attack Source', 'Security Vulnerability Type', 'Defense Mechanism Used']
variaveis_numericas = ['Financial Loss (in Million $)', 'Number of Affected Users', 'Incident Resolution Time (in Hours)']

resultados = []

# Fazendo o teste ANOVA
for cat_var in variaveis_categoricas:
    for num_var in variaveis_numericas:
        grupos = []
        
        for categoria in df[cat_var].unique():
            grupo = df[df[cat_var] == categoria][num_var]
            grupos.append(grupo)
        
        stat, p = f_oneway(*grupos)
        
        significancia = 'Significativo' if p < 0.05 else 'Não significativo'
        resultados.append({
            'Variável Categórica': cat_var,
            'Variável Numérica': num_var,
            'Estatística F': round(stat, 4),
            'p-valor': round(p, 4),
            'Conclusão': significancia
        })

# Criando DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

print(df_resultados)

            Variável Categórica                    Variável Numérica  \
0                       Country        Financial Loss (in Million $)   
1                       Country             Number of Affected Users   
2                       Country  Incident Resolution Time (in Hours)   
3                   Attack Type        Financial Loss (in Million $)   
4                   Attack Type             Number of Affected Users   
5                   Attack Type  Incident Resolution Time (in Hours)   
6               Target Industry        Financial Loss (in Million $)   
7               Target Industry             Number of Affected Users   
8               Target Industry  Incident Resolution Time (in Hours)   
9                 Attack Source        Financial Loss (in Million $)   
10                Attack Source             Number of Affected Users   
11                Attack Source  Incident Resolution Time (in Hours)   
12  Security Vulnerability Type        Financial Loss (in Millio

Não foram encontradas diferenças estatisticamente significativas entre os grupos analisados (p-valores > 0.05 em todos os casos). Isso indica que, dentro do escopo dos dados disponíveis, **as variáveis categóricas não apresentam impacto relevante nas variáveis numéricas estudadas**.

### Associação entre variáveis categóricas

Para mensurar o grau de associação entre variáveis categóricas, utilizei o V de Cramér, que varia de 0 (sem associação) a 1 (associação perfeita). A correção para viés amostral foi aplicada para garantir maior precisão.

In [3]:
# Filtrando apenas colunas categóricas
cat_cols = df.select_dtypes(include='object').columns

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Criar uma matriz V de Cramér
cramers_results = pd.DataFrame(index=cat_cols, columns=cat_cols)

for col1 in cat_cols:
    for col2 in cat_cols:
        cramers_results.loc[col1, col2] = cramers_v(df[col1], df[col2])

# Converter para float
cramers_results = cramers_results.astype(float)

# Plotar o heatmap
fig = px.imshow(cramers_results,
                text_auto=".5f",
                aspect="auto",
                title='Correlação de Pearson entre variáveis quantitativas',
                color_continuous_scale='RdBu_r')

fig.update_layout(
    title_x=0.5  # Centraliza o título
)

fig.show()

Os valores estão todos próximos de 0, o que indica que **não há forte correlação entre as variáveis categóricas** do conjunto de dados.

## Modelagem preditiva

Apesar de o dataset conter diversas informações sobre ataques cibernéticos (tipo de ataque, setor afetado, tempo de resposta, etc.), ele **não apresenta os requisitos mínimos para construção de modelos preditivos eficazes**. Abaixo estão os principais motivos:

### 1. Falta de Relações Estatísticas Relevantes

- Foram aplicados testes como **ANOVA** e **V de Cramér**, que mostraram **baixa ou nenhuma associação estatística significativa** entre variáveis categóricas e variáveis numéricas.
- Isso indica que **as variáveis explicativas não têm poder preditivo suficiente** sobre o impacto financeiro.

---

### 2. Modelos com Baixo Desempenho

- Os modelos de **regressão** (Linear, Random Forest, XGBoost) apresentaram **erros altos**:
  - `MAE > 24 milhões`
  - `RMSE > 28 milhões`
- Na abordagem de **classificação por faixas de impacto (Baixo, Médio, Alto)**:
  - A **acurácia foi de apenas ~34%**, com forte confusão entre as classes.
- Isso demonstra que os algoritmos **não conseguem identificar padrões úteis** nos dados disponíveis.

---

### 3. Ausência de Variáveis Críticas

O impacto financeiro real de um ataque depende de diversos fatores que **não estão presentes na base**, como:

- Tipo de dado vazado (dados pessoais, bancários, sensíveis)
- Porte e receita da empresa
- Penalidades legais e danos à reputação
- Cobertura de seguro cibernético
- Custo de mitigação e recuperação

Essas informações são essenciais para estimar o impacto financeiro com precisão, mas **não fazem parte do conjunto de dados**.

---

### 4. Alto Grau de Aleatoriedade

- Os gráficos de dispersão entre variáveis numéricas **não revelam padrões lineares ou não-lineares**.
- Isso indica um **alto nível de ruído ou variabilidade aleatória**, tornando o problema imprevisível com os dados fornecidos.

---

### Considerações

A base de dados é **adequada para análises descritivas e diagnósticas**. Por outro lado, **não é apropriada para modelagem preditiva de impacto financeiro**, seja por regressão ou classificação.
